# Incremental Ernollment Trial
---------

현재 speaker가 너무 적어서 명수를 늘려서 실험해보려고한다.  
대신 verification set을 따로 떼어놓고 나머지 set에서 만들려고한다.  
현재 최소 150개의 utterance를 가지고 있는 306명을 선정한다.

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('/host/projects/sv_experiments/sv_system')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
from utils import key2df

## Dataframes & embeddings

In [4]:
keys = pickle.load(open("./xvector_embeds/sv_keys.pkl", "rb"))
keys = np.array(keys)
embeds = np.load("./xvector_embeds/sv_embeds.npy")
embeds = embeds / np.linalg.norm(embeds, axis=1, keepdims=True)

In [5]:
key_df = key2df(keys)
key2index = {k:v for v, k in enumerate(keys)}  # it used for mapping from key to index

In [6]:
spk_uttr_stat = key_df.spk.value_counts()

## Held-out validation set

utterance가 150개 넘는 것들을 trial spks로 빼고  
나머지는 validation spk로 빼었다.

In [7]:
trial_enr_spks = spk_uttr_stat[spk_uttr_stat > 150].index.tolist()
val_spks = spk_uttr_stat[spk_uttr_stat <= 150].index.tolist()
val_embed_df = key_df[key_df.spk.isin(val_spks)]
trial_embed_df = key_df[key_df.spk.isin(trial_enr_spks)]

In [9]:
len(val_spks)

945

In [ ]:
voxc2_trial = pd.read_pickle("trials/voxc2_1211_trials.pkl")
trial_for_thresh = voxc2_trial[(voxc2_trial.enroll_spk.isin(val_spks)) & (voxc2_trial.test_spk.isin(val_spks))]

In [ ]:
# os.makedirs("./cases/enr306_uttr1/")
# trial_for_thresh.to_pickle("cases/enr306_uttr1/trial_for_thresh.pkl")
# val_embed_df.to_pickle("./cases/enr306_uttr1/valdiation_df.pkl")

## Design trials for each enr_spk

각 enr_spk당 10번 enr_uttrs 

In [ ]:
n_enroll_utters = 1

In [ ]:
cases = []
for enr_spk in trial_enr_spks:
    target_uttrs = trial_embed_df[trial_embed_df.spk == enr_spk] 
    nonTarget_uttrs = trial_embed_df[trial_embed_df.spk != enr_spk]
    # we have target ratios [0.9, 0.5, 0.1, 0.01]
    nonTarget_n_uttrs = (np.array(len(target_uttrs) -1)*np.array([1/9, 1, 9])).astype(np.int64)
    nonTarget_n_uttrs = np.append(nonTarget_n_uttrs, min(len(nonTarget_uttrs), 99*len(target_uttrs)))
    enr_uttrs = target_uttrs.sample(n=10) # each speaker has 10 enrollment cases
    for idx, enr_uttr in enr_uttrs.iterrows(): # for each enrollment utterance
        for n_imp_uttr in nonTarget_n_uttrs: # neg_trial ratio 
            target_trial_uttrs = target_uttrs.drop(index=idx)
            nonTarget_trial_uttrs = nonTarget_uttrs.sample(n=n_imp_uttr)
            cases += [[enr_spk, 
                       [enr_uttr.name], 
                       target_trial_uttrs.index.tolist(),
                       nonTarget_trial_uttrs.index.tolist()]]


In [ ]:
sorted_cases = sorted(cases, key=lambda x: len(x[3])) 

## Split trials according to pos_ratio

In [ ]:
neg_legnth = list(map(lambda x: len(x[3]), cases))

In [ ]:
pos_ratio_0 = cases[0:len(cases):4] # 0.9
pos_ratio_1 = cases[1:len(cases):4] # 0.5
pos_ratio_2 = cases[2:len(cases):4] # 0.1
pos_ratio_3 = cases[3:len(cases):4] # 0.01

In [ ]:
pickle.dump(pos_ratio_0, open("./cases/enr306_uttr1/trials_ratio_0.9.pkl", "wb"))
pickle.dump(pos_ratio_1, open("./cases/enr306_uttr1/trials_ratio_0.5.pkl", "wb"))
pickle.dump(pos_ratio_2, open("./cases/enr306_uttr1/trials_ratio_0.1.pkl", "wb"))
pickle.dump(pos_ratio_3, open("./cases/enr306_uttr1/trials_ratio_0.01.pkl", "wb"))